In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request, json 
import requests

# Project Goal

Raiding is stressful enough, but loot just dropped and people are fighting/arguing what is BiS (Best in Slot) for them vs someone else -- That makes things REAL stressful!

No one person can be expected to the BiS rankings of <ins>every</ins> item dropped in raid to each cooresponding class/spec! (Not even multiple people can likely fragment that information between them).

##### Goal:
> Create dictionaries that can help Raid leaders/Loot Masters determine, at a glance, who are the BiS users when an item drops in raid

What this code will do:
1. Compile information from various BiS websites
2. Rank items from "1" to "X" based on the respective website's BiS ranking indicator (Ex: wowtbc.gg has "final_value" where a larger value indicates that item being better than another item of the same slot/spec/class)
3. Have 2 dictionaries:
> a) Dicitionary indicating BiS items for a given Class/Spec
> <br>b) Dictionary indicating the class/spec ranking for a given item (reverse of A)
4. Write these dictionaries to `.csv`
5. Create an amazing Google Sheet utilizing this information to allow Raid Leaders + Loot Masters to make educated decisions when it comes to distribution of loot based on their guild's loot rules (Ex: MS > OS, No Loot over Loot)

# wowtbc.gg

Scrap the idea for wowtbc.gg for now --- Injection of "T4" would not refresh the page with the updated info

In [2]:
#Populate the different URLs to access
wowtbc_prefix = r"https://wowtbc.gg/page-data/bis-list/"
wowtbc_suffix = r"/page-data.json"

class_and_specs = {"hunter":['beast-master','survival','marksmanship'],
                   "mage":['fire','arcane','frost'],
                   'priest':['holy','shadow'],
                   'warrior':['arms','fury','protection'],
                   'paladin':['holy','protection','retribution'],
                   "druid":['balance','restoration','feral-dps','feral-tank'],
                   "rogue":['combat','subtlety','assassination'],
                   "shaman":['elemental','enhancement','restoration'],
                   "warlock":['affliction','demonology','destruction','destruction-fire']}

urls = []
for key,val in class_and_specs.items():
    for spec in val:
        urls.append(wowtbc_prefix+spec+"-"+key+wowtbc_suffix)
        
urls

['https://wowtbc.gg/page-data/bis-list/beast-master-hunter/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/survival-hunter/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/marksmanship-hunter/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/fire-mage/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/arcane-mage/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/frost-mage/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/holy-priest/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/shadow-priest/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/arms-warrior/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/fury-warrior/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/protection-warrior/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/holy-paladin/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/protection-paladin/page-data.json',
 'https://wowtbc.gg/page-data/bis-list/retribution-paladin/page-data.json',
 'ht

Code to try and extract the json from the web url below ... it doesn't work ... keeps interpreting the "content-type" as "text/html" when it should be "application/json"

Backup solution -- manually download all json files and load them into memory instead =/

In [3]:
#Make a dictionary of all the Tier pieces and their related conversions
tier_dict = {
        "Helm of the Fallen Champion":[
            'Justicar Crown',
            'Justicar Diadem',
            'Justicar Faceguard',
            'Netherblade Facemask',
            'Cyclone Faceguard',
            'Cyclone Headdress',
            'Cyclone Helm'
        ],
        "Helm of the Fallen Defender":[
            'Warbringer Battle-Helm',
            'Warbringer Greathelm',
            'Light-Collar of the Incarnate',
            'Soul-Collar of the Incarnate',
            'Antlers of Malorne',
            'Crown of Malorne',
            'Stag-Helm of Malorne'
        ],
        "Helm of the Fallen Hero":[
            'Demon Stalker Greathelm',
            'Collar of the Aldor',
            'Voidheart Crown'
        ],
        "Pauldrons of the Fallen Champion":[
            'Justicar Pauldrons',
            'Justicar Shoulderguards',
            'Justicar Shoulderplates',
            'Netherblade Shoulderpads',
            'Cyclone Shoulderpads',
            'Cyclone Shoulderguards',
            'Cyclone Shoulderplates'
        ],
        "Pauldrons of the Fallen Defender":[
            'Warbringer Shoulderguards',
            'Warbringer Shoulderplates',
            'Light-Mantle of the Incarnate',
            'Soul-Mantle of the Incarnate',
            'Mantle of Malorne',
            'Pauldrons of Malorne',
            'Shoulderguards of Malorne'
        ],
        "Pauldrons of the Fallen Hero":[
            'Demon Stalker Shoulderguards',
            'Pauldrons of the Aldor',
            'Voidheart Mantle'
        ],
        "Chestguard of the Fallen Champion":[
            'Justicar Breastplate',
            'Justicar Chestguard',
            'Justicar Chestpiece',
            'Netherblade Chestpiece',
            'Cyclone Breastplate',
            'Cyclone Chestguard',
            'Cyclone Hauberk'
        ],
        "Chestguard of the Fallen Defender":[
            'Warbringer Breastplate',
            'Warbringer Chestguard',
            'Robes of the Incarnate',
            'Shroud of the Incarnate',
            'Breastplate of Malorne',
            'Chestguard of Malorne',
            'Chestpiece of Malorne'
        ],
        "Chestguard of the Fallen Hero":[
            'Demon Stalker Harness',
            'Vestments of the Aldor',
            'Voidheart Robe'
        ],
        "Gloves of the Fallen Champion":[
            'Justicar Gauntlets',
            'Justicar Gloves',
            'Justicar Handguards',
            'Netherblade Gloves',
            'Cyclone Gauntlets',
            'Cyclone Gloves',
            'Cyclone Handguards'
        ],
        "Gloves of the Fallen Defender":[
            'Warbringer Gauntlets',
            'Warbringer Handguards',
            'Gloves of the Incarnate',
            'Handwraps of the Incarnate',
            'Gauntlets of Malorne',
            'Gloves of Malorne',
            'Handguards of Malorne'
        ],
        "Gloves of the Fallen Hero":[
            "Demon Stalker Gauntlets",
            'Gloves of the Aldor',
            'Voidheart Gloves'
        ],
        "Leggings of the Fallen Champion":[
            "Justicar Greaves",
            "Justicar Leggings",
            "Justicar Legguards",
            'Netherblade Breeches',
            'Cyclone Kilt',
            'Cyclone Legguards',
            'Cyclone War-Kilt'
        ],
        "Leggings of the Fallen Defender":[
            "Warbringer Greaves",
            "Warbringer Legguards",
            'Leggings of the Incarnate',
            'Trousers of the Incarnate',
            'Britches of Malorne',
            "Greaves of Malorne",
            'Legguards of Malorne'
        ],
        "Leggings of the Fallen Hero":[
            'Demon Stalker Greaves',
            'Legwraps of the Aldor',
            'Voidheart Leggings'
        ]
    }

#Convert dictionary to pd DataFrame
tier_df = pd.DataFrame({"Token":tier_dict.keys(),"Options":tier_dict.values()})

def tier_piece_items(item):
    '''
    Function to reverse the tier piece back into its Token name
    Ex: "Pauldrons of the Aldor" --> "Pauldrons of the Fallen Hero"
    '''
    token = item['name'] #Initialize token name as the item name
    for i, row in enumerate(tier_df['Options']): #Find item name in tier registry
        if item['name'] in row:
            token = tier_df['Token'][i] #If tier piece, replace token name
            break
    return token

In [4]:
%%time
#Grab all files
import glob
files = glob.glob('wowtbc/*.json')

#Hyperparameters
website = 'wowtbc.gg' #In the future, will add other websites
current_phase = "T4" #Change this in the future to "T5" or "T6"
raids = ['Karazhan',"Gruul's Lair","Magtheridon's Lair"]  # Serpentshrine Cavern, The Eye: Tempest Keep, Hyjal Summit, Zul'Aman


#---Big processing loop
print("Reading files:")
df = pd.DataFrame(columns=["Class","Spec","ItemName","Slot","Rank","Source","Website"])
for file in files:
    with open(file,) as json_file: #Open a file
        print("\t%s"%file)
        #Grab file name to obtain the Class and Spec information
        class_spec_str = file[7:-5].split("-")
        c = class_spec_str[-1]
        spec = ' '.join(class_spec_str[:-1])
        
        data = json.load(json_file) #Load the data as json
        
        #Initialize
        slot_count = 0
        slot_lock = "head"
        
        #Parse each item
        items = data['result']['pageContext']['bisList']
        for item in items:
            if current_phase in item['phase']: #Make sure the item is currently obtainable
                
                #Check item slot because there are some duplicate entries
                if item['slot'] in ['trinket 1','trinket 2']: #Simplify "Trinket1" or "Trinket2" to "trinket"
                    slot = "trinket"
                elif item['slot'] in ['finger 1','finger 2']: #Simplify "finger 1" or "finger 2" to "finger"
                    slot = "finger"
                else:
                    slot = item['slot']                        
                
#                print(slot,slot_lock,slot_count)
                if slot_lock == slot:
                    slot_count += 1 #Increment
                    if slot_lock in ['trinket','finger']:
                        X = 9 #Provide 1 more entries if trinket or finger because you can have 2 each
                    else:
                        X = 8
                    if slot_count <= X: #This limits search to only top X
#                        print("%s registered"%item['name'])
                        #Check if item source 
                        try:
                            if item['source'] == 'Reputation':
                                source = "Rep - " + item['source_type'] #If rep, add faction
                            elif item['source'] == "Quest Reward":
                                source = "Quest - " + item['source_type'] #If quest, add quest name
                            elif item['source'] == 'Vendor':
                                source = item['source_type']
                            else:                            
                                source = item['source']
                        except:
                            source = "Not found"
                        
                        item_name = tier_piece_items(item)
                        
                        d = pd.DataFrame({"Class":[c],
                                          "Spec":[spec],
                                          "ItemName":[item_name],
                                          "Slot":[slot],
                                          "Rank":[slot_count],
                                          "Source": [source],
                                          "Website":[website]})
                        df = df.append(d)
                    else:
                        pass #Don't do anything if slot_count > X
                else:
                    slot_count = 1 #Reset
                    #Check if Slot is "Trinket1" or "Trinket2" and simplify to "trinket"
                    if item['slot'] in ['trinket 1','trinket 2']:
                        slot_lock = "trinket"
                        slot = 'trinket'
                    #Check if Slot is "finger 1" or "finger 2" and simplify to "finger"
                    elif item['slot'] in ['finger 1','finger 2']:
                        slot_lock = "finger"
                        slot = 'finger'
                    else:
                        slot_lock = item['slot'] #Set new lock parameter
                        slot = item['slot']
                    
                    #Check if item source 
                    try:
                        if item['source'] == 'Reputation':
                            source = "Rep - " + item['source_type'] #If rep, add faction
                        elif item['source'] == "Quest Reward":
                            source = "Quest - " + item['source_type'] #If quest, add quest name
                        elif item['source'] == 'Vendor':
                            source = item['source_type']
                        else:                            
                            source = item['source']
                    except:
                        source = "Not found"
                    item_name = tier_piece_items(item)
                    d = pd.DataFrame({"Class":[c],
                                      "Spec":[spec],
                                      "ItemName":[item_name],
                                      "Slot":[slot],
                                      "Rank":[slot_count],
                                      "Source": [source],
                                      "Website":[website]})
                    df = df.append(d)
#                    print("Changing slots to %s"%slot_lock)
#        display(df)        
#    break

df = df.reset_index(drop=True)
df = df.drop_duplicates()

#Write the full registry
df.to_csv("wowtbc/wowtbc_list.csv",index=False) #Full file
print("wowtbc_list.csv written")

#Filter to items from currently existing raids
raid_only = df[df.Source.isin(raids)]
raid_only.to_csv("wowtbc/raid_BiS_wowtbc_list.csv",index=False) #Write kara_only file
print("raid_BiS_wowtbc_list.csv written")

#Creat a dictionary for reverse searching (item --> BiS ranking for all classes/specs)
item_dict = {}
for item in raid_only['ItemName'].unique(): #Find each unique item from raid drops (Because I'm using this during raid only)
    subset = raid_only[raid_only['ItemName']==item] #Find all entries that contain the item
    item_value = []
    for _,row in subset.iterrows(): #Parse each row of the subset
        entry = row['Spec']+"-"+row['Class']+": "+str(row['Rank']) #Combine the Spec+Class+Rank together
        item_value.append(entry) #Append
    item_dict[item] = item_value #Write the value into dictionary
    
#Write the file 
item_dict_df = pd.DataFrame({"ItemName":item_dict.keys() #Convert to pd Dataframe
             ,"Bis_Slots":item_dict.values()})

item_dict_df = item_dict_df.sort_values(by=['ItemName'])

item_dict_df.to_csv("wowtbc/BiS_dictionary_wowtbc.csv",index=False) #Write
print("Dictionary written")

Reading files:
	wowtbc\affliction-warlock.json
	wowtbc\arcane-mage.json
	wowtbc\arms-warrior.json
	wowtbc\assassination-rogue.json
	wowtbc\balance-druid.json
	wowtbc\beast-master-hunter.json
	wowtbc\combat-rogue.json
	wowtbc\demonology-warlock.json
	wowtbc\destruction-fire-warlock.json
	wowtbc\destruction-warlock.json
	wowtbc\elemental-shaman.json
	wowtbc\enhancement-shaman.json
	wowtbc\feral-dps-druid.json
	wowtbc\feral-tank-druid.json
	wowtbc\fire-mage.json
	wowtbc\frost-mage.json
	wowtbc\fury-warrior.json
	wowtbc\holy-paladin.json
	wowtbc\holy-priest.json
	wowtbc\marksmanship-hunter.json
	wowtbc\protection-paladin.json
	wowtbc\protection-warrior.json
	wowtbc\restoration-druid.json
	wowtbc\restoration-shaman.json
	wowtbc\retribution-paladin.json
	wowtbc\shadow-priest.json
	wowtbc\subtlety-rogue.json
	wowtbc\survival-hunter.json
wowtbc_list.csv written
raid_BiS_wowtbc_list.csv written
Dictionary written
Wall time: 4.27 s


In [5]:
def lookup(CLASS,spec,slot):    
    '''
    Function to do a quick search of BiS by class, spec, and slot
    '''
    s = df[df['Class']==CLASS][df[df['Class']==CLASS]["Spec"]==spec]
    return s[s['Slot']==slot]

lookup('mage','arcane','ranged')

,Class,Spec,ItemName,Slot,Rank,Source,Website
236,mage,arcane,Tirisfal Wand of Ascendancy,ranged,1,Karazhan,wowtbc.gg
237,mage,arcane,Nether Core's Control Rod,ranged,2,Arcatraz,wowtbc.gg
238,mage,arcane,Eredar Wand of Obliteration,ranged,3,Magtheridon's Lair,wowtbc.gg
239,mage,arcane,Voidfire Wand,ranged,4,Mana-Tombs,wowtbc.gg
240,mage,arcane,Rod of Dire Shadows,ranged,5,Quest - The Will of the Warchief,wowtbc.gg
241,mage,arcane,Nexus Torch,ranged,6,Shattered Halls,wowtbc.gg
242,mage,arcane,Gladiator's Touch of Defeat,ranged,7,PvP,wowtbc.gg
243,mage,arcane,Wand of the Seer,ranged,8,Quest - Turning Point (Scryers),wowtbc.gg


In [6]:
def bis(CLASS,spec):
    '''
    Function to find the BiS for all slots for a given Class + Spec
    '''
    slots = ['head','neck','shoulder','back','chest','hands','wrist','waist','legs','feet',
             'weapon','off hand','ranged','finger 1','finger 2','trinket 1',' trinket 2']
    
    for slot in slots:
        index = 0
        if "2" in slot:
            index = 1
        if 'trinket' in slot:
            slot = 'trinket'
        elif 'finger' in slot:
            slot = 'finger'

        slot_bis = lookup(CLASS,spec,slot).iloc[index]['ItemName']
        print("%s: %s"%(slot.capitalize(),slot_bis))

bis('priest','holy')

Head: Helm of the Fallen Defender
Neck: Archaic Charm of Presence
Shoulder: Pauldrons of the Fallen Defender
Back: Stainless Cloak of the Pure Hearted
Chest: Primal Mooncloth Robe
Hands: Gloves of Saintly Blessings
Wrist: Bands of Indwelling
Waist: Primal Mooncloth Belt
Legs: Gilded Trousers of Benediction
Feet: Boots of the Incorrupt
Weapon: Light's Justice
Off hand: Windcaller's Orb
Ranged: Blue Diamond Witchwand
Finger: Ring of Flowing Light
Finger: Violet Signet of the Grand Restorer
Trinket: Essence of the Martyr
Trinket: Ribbon of Sacrifice


In [7]:
def reverse_lookup(item_name):
    return item_dict[item_name]

reverse_lookup('Eredar Wand of Obliteration')

['affliction-warlock: 2',
 'arcane-mage: 3',
 'demonology-warlock: 3',
 'destruction fire-warlock: 2',
 'destruction-warlock: 3',
 'fire-mage: 2',
 'frost-mage: 2',
 'holy-priest: 2',
 'shadow-priest: 7']

# WowHead
Brainstorming section to play with WoWhead's registries